# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

A newer version of GraphLab Create (v1.8.5) is available! Your current version is v1.8.3.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [12]:
sales = graphlab.SFrame('kc_house_data.gl/')
import numpy as np

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Find what features had non-zero weight.

In [7]:
model_all.get('coefficients').print_rows(20)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [8]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [30]:
smallest_l1_penalty = 0
previous_rss = 0
for l1_penalty in np.logspace(1, 7, num=13):
    model_yep = graphlab.linear_regression.create(training, features=all_features,
                                              target='price', l2_penalty=0.,l1_penalty=l1_penalty,
                                              validation_set=None,verbose=False)
    rss = ((model_yep.predict(validation) - validation['price'])**2).sum()
    
    print 'l1_penalty: ' +  str(l1_penalty)
    print "{:,}".format(rss)
    
    if rss < previous_rss or previous_rss == 0:
        smallest_l1_penalty = l1_penalty
        print "smallest -> " + str(smallest_l1_penalty)
        
    print 'number of nonzeros = %d' % (model_yep.coefficients['value']).nnz()
    previous_rss = rss
        
print "--------"


l1_penalty: 10.0
6.25766285142e+14
smallest -> 10.0
number of nonzeros = 18
l1_penalty: 31.6227766017
6.25766285362e+14
number of nonzeros = 18
l1_penalty: 100.0
6.25766286058e+14
number of nonzeros = 18
l1_penalty: 316.227766017
6.25766288257e+14
number of nonzeros = 18
l1_penalty: 1000.0
6.25766295212e+14
number of nonzeros = 18
l1_penalty: 3162.27766017
6.25766317206e+14
number of nonzeros = 18
l1_penalty: 10000.0
6.25766386761e+14
number of nonzeros = 18
l1_penalty: 31622.7766017
6.25766606749e+14
number of nonzeros = 18
l1_penalty: 100000.0
6.25767302792e+14
number of nonzeros = 18
l1_penalty: 316227.766017
6.25769507644e+14
number of nonzeros = 18
l1_penalty: 1000000.0
6.25776517727e+14
number of nonzeros = 18
l1_penalty: 3162277.66017
6.25799062845e+14
number of nonzeros = 18
l1_penalty: 10000000.0
6.25883719085e+14
number of nonzeros = 18
--------


In [31]:
new_model = graphlab.linear_regression.create(training, features=all_features,
                                              target='price', l2_penalty=0.,l1_penalty=10000000,
                                              validation_set=None,verbose=False)
new_model.get('coefficients').print_rows(20)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None |   19604.7163508    |  None  |
|     bedrooms     |  None |   7922.62023075    |  None  |
| bedrooms_square  |  None |   928.112147889    |  None  |
|    bathrooms     |  None |    25392.743443    |  None  |
|   sqft_living    |  None |   39.1083048767    |  None  |
| sqft_living_sqrt |  None |   1123.95233925    |  None  |
|     sqft_lot     |  None | -0.000823987151992 |  None  |
|  sqft_lot_sqrt   |  None |   147.859054391    |  None  |
|      floors      |  None |    21154.353574    |  None  |
|  floors_square   |  None |    12871.042877    |  None  |
|    waterfront    |  None |   598538.378522    |  None  |
|       view       |  None |   93089.3048849    |  None  |
|    condition     |  None |   6584.64358167    |  None  |
|      grade       |  None |   6200.37319739    |  None 

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [32]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [33]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [36]:
for l1_penalty in l1_penalty_values:
    model_STUPID = graphlab.linear_regression.create(training, features=all_features,
                                              target='price', l2_penalty=0.,l1_penalty=l1_penalty,
                                              validation_set=None,verbose=False)
    rss = ((model_STUPID.predict(validation) - validation['price'])**2).sum()
    
    print 'l1_penalty: ' + str(l1_penalty)
    print  model_STUPID['coefficients']['value'].nnz()
    
    print "--------"


l1_penalty: 100000000.0
18
l1_penalty: 127427498.57
18
l1_penalty: 162377673.919
18
l1_penalty: 206913808.111
18
l1_penalty: 263665089.873
17
l1_penalty: 335981828.628
17
l1_penalty: 428133239.872
17
l1_penalty: 545559478.117
17
l1_penalty: 695192796.178
17
l1_penalty: 885866790.41
16
l1_penalty: 1128837891.68
15
l1_penalty: 1438449888.29
15
l1_penalty: 1832980710.83
13
l1_penalty: 2335721469.09
12
l1_penalty: 2976351441.63
10
l1_penalty: 3792690190.73
6
l1_penalty: 4832930238.57
5
l1_penalty: 6158482110.66
3
l1_penalty: 7847599703.51
1
l1_penalty: 10000000000.0
1
--------


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [40]:
l1_penalty_min = 2976351441.63
l1_penalty_max = 3792690190.73

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [41]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [42]:
for l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
    model_STUPID = graphlab.linear_regression.create(training, features=all_features,
                                              target='price', l2_penalty=0.,l1_penalty=l1_penalty,
                                              validation_set=None,verbose=False)
    rss = ((model_STUPID.predict(validation) - validation['price'])**2).sum()
    
    print 'l1_penalty: ' + str(l1_penalty)
    print rss
    
    print  model_STUPID['coefficients']['value'].nnz()
    
    print "--------"


l1_penalty: 2976351441.63
9.66925692362e+14
10
--------
l1_penalty: 3019316638.95
9.74019450085e+14
10
--------
l1_penalty: 3062281836.27
9.81188367942e+14
10
--------
l1_penalty: 3105247033.59
9.89328342459e+14
10
--------
l1_penalty: 3148212230.91
9.98783211266e+14
10
--------
l1_penalty: 3191177428.24
1.00847716702e+15
10
--------
l1_penalty: 3234142625.56
1.01829878055e+15
10
--------
l1_penalty: 3277107822.88
1.02824799221e+15
10
--------
l1_penalty: 3320073020.2
1.03461690923e+15
8
--------
l1_penalty: 3363038217.52
1.03855473594e+15
8
--------
l1_penalty: 3406003414.84
1.04323723787e+15
8
--------
l1_penalty: 3448968612.16
1.04693748875e+15
7
--------
l1_penalty: 3491933809.48
1.05114762561e+15
7
--------
l1_penalty: 3534899006.8
1.05599273534e+15
7
--------
l1_penalty: 3577864204.12
1.06079953176e+15
7
--------
l1_penalty: 3620829401.45
1.0657076895e+15
6
--------
l1_penalty: 3663794598.77
1.06946433543e+15
6
--------
l1_penalty: 3706759796.09
1.07350454959e+15
6
--------
l1_pe

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [44]:
model_STUPID = graphlab.linear_regression.create(training, features=all_features,
                                              target='price', l2_penalty=0.,l1_penalty=3448968612.16,
                                              validation_set=None,verbose=False)



In [45]:
model_STUPID.get("coefficients")

name,index,value,stderr
(intercept),None,222253.192544,None
bedrooms,None,661.722717782,None
bedrooms_square,None,0.0,None
bathrooms,None,15873.9572593,None
sqft_living,None,32.4102214513,None
sqft_living_sqrt,None,690.114773313,None
sqft_lot,None,0.0,None
sqft_lot_sqrt,None,0.0,None
floors,None,0.0,None
floors_square,None,0.0,None
